In [37]:
%autosave 10

Autosaving every 10 seconds



<h1 align="center">Resilient Distributed Datasets</h1>



# Table of Contents

- [I. Internal Workings of an RDD](#Internal-Workings-of-an-RDD)
- [II. Creating RDDs](#Creating-RDDs)
- [III. Transformations](#Transformations)
- [IV. Actions](#Actions)
- [V. Spark 2.0 architecture](#Spark-2.0-architecture)

  


RDDs are a distributed collection of immutable JVM objects that allow you to perform calculations very quickly, and they are the **backbone** of Apache spark.

- As the name suggests, the dataset is distributed; it is split into chunks based on some key and distributed to executor nodes. 

- RDDs keep track(log) of all the transformations applied to each chunk to speed up the computations and provide a fallback if things go wrong and that portion of the data is lost; in such cases, RDDs can recompute the data.


# Internal workings of an RDD

RDDs operate in parallel. This is the strongest advantage of working in Spark: Each transformation is executed in parallel for enormous increase in speed.

The transformations to the dataset are lazy. This means that any transformation is only executed when an action on a dataset is called. This helps Spark to optimize the execution. For instance, consider the following very common steps that an analyst would normally do to get familiar with a dataset:

1. Count the occurrence of distinct values in a certain column.
2. Select those that start with an A.
3. Print the results to the screen.

As simple as the previously mentioned steps sound, if only items that start with the letter A are of interest, there is no point in counting distinct values for all the other items. Thus, instead of following the execution as outlined in the preceding points, Spark could only count the items that start with A, and then print the results to the screen.

Let's break this example down in code. First, we order Spark to map the values of A using the ```.map(lambda v: (v, 1))``` method, and then select those records that start with an 'A' (using the ```.filter(lambda val: val.startswith('A'))``` method). If we call the ```.reduceByKey(operator.add)``` method it will reduce the dataset and add (in this example, count) the number of occurrences of each key. All of these steps transform the dataset.

Second, we call the ```.collect()``` method to execute the steps. This step is an action on our dataset - it finally counts the distinct elements of the dataset. In effect, the action might reverse the order of transformations and filter the data first before mapping, resulting in a smaller dataset being passed to the reducer.

# Creating RDDs
[back to top](#Table-of-Contents)

There are two ways to create an RDD in PySpark. You can either
- ```.parallelize(...)``` a colection(list or an array of some elements):


In [1]:
data = sc.parallelize(
          [('Amber', 22), ('Alfred', 23), ('Skye',4), 
           ('Albert', 12), ('Amber', 9)])

- or you can reference a file(or files located either locally or somewhere externally)

In [15]:
file = "data/kddcup2008/VS14MORT.txt.gz"
data_From_file = sc.textFile(file, 4)

In [16]:
data_From_file.take(1)

['                   1                                          2101  M1087 432311  4M4                2014U7CN                                    I64 238 070   24 0111I64                                                                                                                                                                           01 I64                                                                                                  01  11                                 100 601']

The last parameter in ```sc.textFile(..., n)``` specifies the number of partitions the dataset is divided into.

**TIP: A rule of thumb would be to break your dataset into two-four partitions for each in your cluster.**



## Schema

RDDs are *schema-less* data structures(unlike DataFrames). Thus, parallelizing a dataset, such as in the following snippet, is perfectly fine with Spark when using RDDs:


In [17]:
data_heterogenous = sc.parallelize([('Ferrari', 'fast'),   
                                    {'Porsche': 100000},   
                                    ['Spain','visited', 4504]]).collect()

So, we can mix almost anything:
- tuple
- dict
- list

and Spark wil not complain.

Once you ```.collect()``` the dataset, you can access the data in the object.

In [18]:
data_heterogenous[1]

{'Porsche': 100000}

## Reading from files
[back to top](#Table-of-Contents)

When reading from a text file, each row from the file forms an element of an RDD.

In [19]:
data_From_file.take(2)

['                   1                                          2101  M1087 432311  4M4                2014U7CN                                    I64 238 070   24 0111I64                                                                                                                                                                           01 I64                                                                                                  01  11                                 100 601',
 '                   1                                          2101  M1058 371708  4D3                2014U7CN                                    I250214 062   21 0311I250 61I272 62E669                                                                                                                                                            03 I250 E669 I272                                                                                       01  11                                 100 601']

## User defined functions

We can create **longer** method to transform our data instead of using the lambda expression.

In [20]:
def extractInformation(row):
    import re
    import numpy as np

    selected_indices = [
         2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,
         19,21,22,23,24,25,27,28,29,30,32,33,34,
         36,37,38,39,40,41,42,43,44,45,46,47,48,
         49,50,51,52,53,54,55,56,58,60,61,62,63,
         64,65,66,67,68,69,70,71,72,73,74,75,76,
         77,78,79,81,82,83,84,85,87,89
    ]

    '''
        Input record schema
        schema: n-m (o) -- xxx
            n - position from
            m - position to
            o - number of characters
            xxx - description
        1. 1-19 (19) -- reserved positions
        2. 20 (1) -- resident status
        3. 21-60 (40) -- reserved positions
        4. 61-62 (2) -- education code (1989 revision)
        5. 63 (1) -- education code (2003 revision)
        6. 64 (1) -- education reporting flag
        7. 65-66 (2) -- month of death
        8. 67-68 (2) -- reserved positions
        9. 69 (1) -- sex
        10. 70 (1) -- age: 1-years, 2-months, 4-days, 5-hours, 6-minutes, 9-not stated
        11. 71-73 (3) -- number of units (years, months etc)
        12. 74 (1) -- age substitution flag (if the age reported in positions 70-74 is calculated using dates of birth and death)
        13. 75-76 (2) -- age recoded into 52 categories
        14. 77-78 (2) -- age recoded into 27 categories
        15. 79-80 (2) -- age recoded into 12 categories
        16. 81-82 (2) -- infant age recoded into 22 categories
        17. 83 (1) -- place of death
        18. 84 (1) -- marital status
        19. 85 (1) -- day of the week of death
        20. 86-101 (16) -- reserved positions
        21. 102-105 (4) -- current year
        22. 106 (1) -- injury at work
        23. 107 (1) -- manner of death
        24. 108 (1) -- manner of disposition
        25. 109 (1) -- autopsy
        26. 110-143 (34) -- reserved positions
        27. 144 (1) -- activity code
        28. 145 (1) -- place of injury
        29. 146-149 (4) -- ICD code
        30. 150-152 (3) -- 358 cause recode
        31. 153 (1) -- reserved position
        32. 154-156 (3) -- 113 cause recode
        33. 157-159 (3) -- 130 infant cause recode
        34. 160-161 (2) -- 39 cause recode
        35. 162 (1) -- reserved position
        36. 163-164 (2) -- number of entity-axis conditions
        37-56. 165-304 (140) -- list of up to 20 conditions
        57. 305-340 (36) -- reserved positions
        58. 341-342 (2) -- number of record axis conditions
        59. 343 (1) -- reserved position
        60-79. 344-443 (100) -- record axis conditions
        80. 444 (1) -- reserve position
        81. 445-446 (2) -- race
        82. 447 (1) -- bridged race flag
        83. 448 (1) -- race imputation flag
        84. 449 (1) -- race recode (3 categories)
        85. 450 (1) -- race recode (5 categories)
        86. 461-483 (33) -- reserved positions
        87. 484-486 (3) -- Hispanic origin
        88. 487 (1) -- reserved
        89. 488 (1) -- Hispanic origin/race recode
     '''

    record_split = re\
        .compile(
            r'([\s]{19})([0-9]{1})([\s]{40})([0-9\s]{2})([0-9\s]{1})([0-9]{1})([0-9]{2})' + 
            r'([\s]{2})([FM]{1})([0-9]{1})([0-9]{3})([0-9\s]{1})([0-9]{2})([0-9]{2})' + 
            r'([0-9]{2})([0-9\s]{2})([0-9]{1})([SMWDU]{1})([0-9]{1})([\s]{16})([0-9]{4})' +
            r'([YNU]{1})([0-9\s]{1})([BCOU]{1})([YNU]{1})([\s]{34})([0-9\s]{1})([0-9\s]{1})' +
            r'([A-Z0-9\s]{4})([0-9]{3})([\s]{1})([0-9\s]{3})([0-9\s]{3})([0-9\s]{2})([\s]{1})' + 
            r'([0-9\s]{2})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([\s]{36})([A-Z0-9\s]{2})([\s]{1})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([\s]{1})([0-9\s]{2})([0-9\s]{1})' + 
            r'([0-9\s]{1})([0-9\s]{1})([0-9\s]{1})([\s]{33})([0-9\s]{3})([0-9\s]{1})([0-9\s]{1})')
    try:
        rs = np.array(record_split.split(row))[selected_indices]
    except:
        rs = np.array(['-99'] * len(selected_indices))
    return rs
#     return record_split.split(row)

Now, instead of using ```lambda``` we will use the ```extractInformation(...)``` method to split and convert our dataset.

In [21]:
data_from_file_conv = data_From_file.map(extractInformation)

In [22]:
data_from_file_conv.map(lambda row: row).take(1)

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], 
       dtype='<U40')]

In [23]:
data_from_file_conv.take(1)

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], 
       dtype='<U40')]

## Transformations
[back to top](#Table-of-Contents)

### ```.map(...)```

This method is applied to each element of the RDD: in the case for ```data_from_file_conv``` dataset we can think of this as a transformation of each row.

In [25]:
data_2014 = data_from_file_conv.map(lambda row: int(row[16]))
data_2014.take(10)

[2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, -99]

We can combine more columns

In [26]:
data_2014_2 = data_from_file_conv.map(lambda row: (row[16], int(row[16])))
data_2014_2.take(10)

[('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('-99', -99)]

### ```.filter(...)```
The ```.filter(...) method allows you to select elements of our dataset that fit specified criteria.


In [27]:
data_filtered = data_from_file_conv.filter(lambda row: row[5] == 'F' 
                                          and row[21] == '0')

In [29]:
data_filtered.take(1)

[array(['2', '12', ' ', '0', '07', 'F', '1', '030', ' ', '32', '12', '05',
        '  ', '1', 'D', '6', '2014', 'N', '1', 'U', 'Y', '0', '9', 'X44 ',
        '420', '122', '   ', '39', '05', '11T391 ', '12X44  ', '13T401 ',
        '14T424 ', '61F199 ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '05',
        'X44  ', 'F199 ', 'T391 ', 'T401 ', 'T424 ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], 
       dtype='<U40')]

In [30]:
data_filtered.count()

6

### ```.flatMap(...)```
The ```.flatMap(...)``` method works similar to ```.map(...)``` but returns a flattened results instead of a list.

In [32]:
data_2014_flat = data_from_file_conv.flatMap(lambda row: (row[16], 
                                                          int(row[16]) + 1))
data_2014_flat.take(10)

['2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015]

### ```.distinct(...)```
[back to top](#Table-of-Contents)

This method returns a list of distinct values in a specified column.


In [34]:
distinct_gender = data_from_file_conv.map(lambda row: row[5]).distinct().collect()
distinct_gender

['-99', 'M', 'F']

### ```sample(...)```

The ```.sample()``` method returns a randomized sample from the dataset.

In [35]:
fraction = 0.1
data_sample = data_from_file_conv.sample(False, fraction, 666)
data_sample.take(1)

[array(['1', '  ', '5', '1', '01', 'F', '1', '082', ' ', '42', '22', '10',
        '  ', '4', 'W', '5', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I251',
        '215', '063', '   ', '21', '02', '11I350 ', '21I251 ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '02',
        'I251 ', 'I350 ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '28', ' ',
        ' ', '2', '4', '100', '8'], 
       dtype='<U40')]

Let's confirm that we really got 10% of all the records.

In [36]:
print('Original dataset: {0}, sample: {1}'.format(data_from_file_conv.count(), 
                                                  data_sample.count()))

Original dataset: 2631171, sample: 263247


### ```.leftOuterJoin(...)```

Left outer join, just like the SQL world, joins two RDDs based on values found in both datasets, and returns records from the left RDD with records from the right one appended where the two RDDs match.


In [42]:
rdd1 = sc.parallelize([('a', 1), ('b', 4), ('c',10)])
rdd2 = sc.parallelize([('a', 4), ('a', 1), ('b', 6), ('d', 15)])

In [43]:
rdd3 = rdd1.leftOuterJoin(rdd2)
rdd3.take(5)

[('b', (4, 6)), ('a', (1, 4)), ('a', (1, 1)), ('c', (10, None))]

If we used ```.join(...)``` method instead we would have gotten only the values for 'a' and 'b' as these two values intersect between these two RDDs.

In [44]:
rdd4 = rdd1.join(rdd2)
rdd4.collect()

[('b', (4, 6)), ('a', (1, 4)), ('a', (1, 1))]

Another useful method is the ```.intersection(...)``` that returns the records that are equal in both RDDs.

In [45]:
rdd5 = rdd1.intersection(rdd2)
rdd5.collect()

[('a', 1)]

### ```repartition(...)```

Repartitioning the dataset changes the number of partitions the dataset is divided into.



In [46]:
len(rdd1.glom().collect())

16

In [47]:
rdd1 = rdd1.repartition(4)
len(rdd1.glom().collect())

4

## Actions

[back to top](#Table-of-Contents)

### ```.take(...)```
The method returns n top rows from a single data partition


In [48]:
data_first = data_from_file_conv.take(1)
data_first

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], 
       dtype='<U40')]


If you want somewhat randomized records you can use ```.takeSample(...)``` instead.

In [49]:
data_take_sampled = data_from_file_conv.takeSample(False, 1, 667)
data_take_sampled

[array(['2', '17', ' ', '0', '08', 'M', '1', '069', ' ', '39', '19', '09',
        '  ', '1', 'M', '7', '2014', 'U', '7', 'U', 'N', ' ', ' ', 'I251',
        '215', '063', '   ', '21', '06', '11I500 ', '21I251 ', '61I499 ',
        '62I10  ', '63N189 ', '64K761 ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '05',
        'I251 ', 'I120 ', 'I499 ', 'I500 ', 'K761 ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], 
       dtype='<U40')]

### ```.reduce(...)```

Another action that processes your data, the ```.reduce(...)``` method **reduces** the elements of an RDD using a specified method.

In [50]:
rdd1.collect()

[('a', 1), ('b', 4), ('c', 10)]

In [51]:
rdd1.map(lambda row: row[1]).reduce(lambda x, y: x + y)

15

If the reducing function is not associative and commutative you will sometimes get wrong results depending how your data is partitioned.


In [56]:
data_reduce = sc.parallelize([1, 2, .5, .1, 5, .2], 1)

If we were to reduce the data in a manner that we would like to divide the current result by the subsequent one, we would expect a value of 10

In [57]:
works = data_reduce.reduce(lambda x, y: x/y )
works

10.0

However, if we were to partition the data into 3 partitions, the result will be wrong

In [58]:
data_reduce = sc.parallelize([1, 2, .5, .1, 5, .2], 3)
data_reduce.reduce(lambda x, y: x / y)

0.004

The ```.reduceByKey(...)``` method works in a similar way to the ```.reduce(...)``` method but performs a reduction on a key-by-key basis.

In [59]:
data_key = sc.parallelize([('a', 4),('b', 3),('c', 2),('a', 8),('d', 2),('b', 1),('d', 3)],4)
data_key.reduceByKey(lambda x, y: x + y).collect()

[('b', 4), ('c', 2), ('a', 12), ('d', 5)]

### ```count(...)```

The ```.count(..)``` method counts the number of elements in the RDD.

In [60]:
data_reduce.count()

6

It has the same effect as the method below but does not require shifting the data to the driver.

In [61]:
len(data_reduce.collect()) # WRONG -- DON'T DO THIS!

6

If your dataset is in a form of a key-value you can use the ```.countByKey()``` method to get the counts of distinct keys.

### ```saveAsTextFile(...)```

As the name suggests, the ```.saveAsTextFile()``` the RDD and saves it to text files: each partition to a separate file.


In [62]:
data_key.saveAsTextFile("data/data_key.txt")

To read it back, we need to parse it back as before, all the rows are treated as strings.

In [64]:
def parseInput(row):
    import re
    
    pattern = re.compile(r'\(\'([a-z])\', ([0-9])\)')
    row_split = pattern.split(row)
    
    return (row_split[1], int(row_split[2]))
    
data_key_reread = sc.textFile('data/data_key.txt').map(parseInput)
data_key_reread.collect()

[('a', 4), ('b', 1), ('d', 3), ('a', 8), ('d', 2), ('b', 3), ('c', 2)]

### ```foreach(...)```
A method that applies the same function to each element of the RDD in an iterative way.

In [65]:
def f(x): 
    print(x)

data_key.foreach(f)